<a href="https://colab.research.google.com/github/bintangnabiil/Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow/blob/main/Rangkuman_Chapter_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 16: Natural Language Processing with RNNs and Attention
Chapter ini membahas penerapan Recurrent Neural Networks (RNNs) dan mekanisme Attention untuk Natural Language Processing (NLP). Chapter ini merupakan kelanjutan dari Chapter 15 yang fokus pada aplikasi khusus untuk memproses bahasa natural.

Natural Language Processing adalah cabang dari artificial intelligence yang berfokus pada interaksi antara komputer dan bahasa manusia. Tantangan utama dalam NLP adalah bahasa manusia memiliki karakteristik yang kompleks seperti ambiguitas, konteks yang berubah, idiom, dan struktur gramatikal yang beragam. RNN dan mekanisme attention menjadi solusi yang efektif karena mampu menangkap dependensi temporal dan konteks yang panjang dalam teks.
<br><br>

###Perbedaan utama antara pemrosesan teks dan data sekuensial lainnya adalah:

- Diskretisasi: Teks terdiri dari token diskret (kata, karakter) bukan nilai kontinu
- Variabilitas panjang: Kalimat memiliki panjang yang sangat bervariasi
- Konteks semantik: Makna kata bergantung pada konteks sekitarnya
- Struktur hierarkis: Bahasa memiliki struktur dari karakter → kata → frasa → kalimat → paragraf

##1. Character-Level Language Model
Language model karakter-level memprediksi karakter berikutnya dalam sebuah teks berdasarkan karakter-karakter sebelumnya. Ini adalah pendekatan dasar dalam text generation.
<br><br>

###Character-level language model memiliki beberapa keuntungan:

- Vocabulary terbatas: Hanya perlu menangani ~100 karakter unik
- No OOV problem: Tidak ada masalah Out-of-Vocabulary karena semua kata dapat dibentuk dari karakter
- Multilingual: Dapat menangani berbagai bahasa dengan arsitektur yang sama
- Morfologi: Dapat mempelajari morphological patterns secara implisit
<br><br>

###Tantangan Character-Level:

- Long sequences: Teks menjadi sangat panjang dalam representasi karakter
- Long-term dependencies: Sulit menangkap dependensi antar kata yang berjauhan
- Computational cost: Membutuhkan komputasi yang lebih besar
- Slower convergence: Training membutuhkan waktu lebih lama
<br><br>

###Prinsip Kerja:
Model character-level bekerja dengan prinsip autoregressive, di mana prediksi karakter ke-t bergantung pada semua karakter sebelumnya: P(c_t | c_1, c_2, ..., c_{t-1}). Dalam praktiknya, RNN mempertahankan hidden state yang merangkum informasi dari karakter-karakter sebelumnya.
<br><br>

###Aplikasi:

- Text generation (seperti GPT awal)
- Text completion
- Style transfer
- Language modeling untuk compression

###Implementasi Character-Level RNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import requests

# Download dataset (contoh: Shakespeare text)
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

print(f"Length of text: {len(shakespeare_text)} characters")
print(shakespeare_text[:100])

###Preprocessing Text Data

In [ ]:
# Buat vocabulary dan encoding
text = shakespeare_text
vocab = sorted(set(text))
print(f"Unique characters: {len(vocab)}")

# Character to integer mapping
char_to_ids = {char: i for i, char in enumerate(vocab)}
ids_to_char = {i: char for i, char in enumerate(vocab)}

# Convert text to integer sequences
encoded = np.array([char_to_ids[char] for char in text])

# Create training sequences
seq_length = 100
examples_per_epoch = len(text) // seq_length

# Convert to tf.data.Dataset
char_dataset = tf.data.Dataset.from_tensor_slices(encoded)

# Batch the characters into sequences
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# Create input-target pairs
dataset = sequences.map(split_input_target)

# Shuffle and batch
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (dataset
          .shuffle(BUFFER_SIZE)
          .batch(BATCH_SIZE, drop_remainder=True)
          .prefetch(tf.data.experimental.AUTOTUNE))

# Contoh data
for input_example, target_example in dataset.take(1):
    print(f'Input shape: {input_example.shape}')
    print(f'Target shape: {target_example.shape}')

###Building Character-Level Model

In [ ]:
def build_char_rnn_model(vocab_size, embedding_dim=256, rnn_units=1024, batch_size=None):
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        keras.layers.Dense(vocab_size)
    ])
    return model

# Build and compile model
vocab_size = len(vocab)
model = build_char_rnn_model(vocab_size, batch_size=BATCH_SIZE)

# Define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)
model.summary()

###Training the Character Model

In [ ]:
# Configure checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = checkpoint_dir + "/ckpt_{epoch}"

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Train the model
EPOCHS = 10
history = model.fit(dataset,
                   epochs=EPOCHS,
                   callbacks=[checkpoint_callback])

###Text Generation

In [ ]:
# Build model for generation (batch_size=1)
generation_model = build_char_rnn_model(vocab_size, batch_size=1)
generation_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
generation_model.build(tf.TensorShape([1, None]))

def generate_text(model, start_string, generation_length=1000, temperature=1.0):
    # Convert start string to numbers
    input_eval = [char_to_ids[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    model.reset_states()

    for i in range(generation_length):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # Apply temperature
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass predicted character as input for next time step
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(ids_to_char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate text
print(generate_text(generation_model, start_string="ROMEO: ", temperature=0.5))

##2. Sentiment Analysis dengan LSTM

###Teori Sentiment Analysis
Sentiment analysis atau opinion mining adalah tugas klasifikasi teks yang bertujuan menentukan polaritas emosi dalam teks (positif, negatif, atau netral). Ini adalah salah satu aplikasi NLP yang paling populer dengan berbagai use case komersial.
<br><br>

###Tantangan dalam Sentiment Analysis:

- Konteks: Kata yang sama dapat memiliki sentimen berbeda dalam konteks berbeda
- Sarkasme dan ironi: Sulit dideteksi oleh model
- Negasi: Kata negasi dapat membalik sentimen keseluruhan
- Intensitas: Perbedaan antara "good" dan "excellent"
- Domain dependence: Model yang trained pada review film mungkin tidak baik untuk review produk
<br><br>

###Mengapa LSTM Efektif untuk Sentiment Analysis:

- Sequential processing: Dapat memproses kata secara berurutan
- Long-term memory: Mengingat konteks dari awal kalimat
- Gating mechanism: Dapat memfilter informasi yang tidak relevan
- Bidirectional capability: Dapat mempertimbangkan konteks masa depan
<br><br>

###Arsitektur Umum:

- Embedding layer: Mengkonversi kata menjadi dense vector
- LSTM layer: Memproses sequence dan menangkap dependencies
- Dense layer: Klasifikasi final dengan sigmoid/softmax activation
- Dropout: Regularization untuk mencegah overfitting
<br><br>

###Preprocessing Khusus untuk Sentiment:

- Handling negations: Menggabungkan kata negasi dengan kata berikutnya
- Emoticons: Mengkonversi emoticon menjadi kata
- Normalization: Mengubah repeated characters (gooood → good)
- Stop words: Biasanya tidak dihilangkan karena dapat mempengaruhi sentimen

###Persiapan Data untuk Sentiment Analysis

In [ ]:
# Menggunakan IMDB dataset
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# Load IMDB dataset
max_features = 10000  # Only consider top 10k words
maxlen = 500  # Cut texts after this number of words

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(f'Train sequences: {len(x_train)}')
print(f'Test sequences: {len(x_test)}')

# Pad sequences
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')

###LSTM Model for Sentiment Analysis

In [ ]:
def build_sentiment_model(max_features, maxlen):
    model = keras.Sequential([
        keras.layers.Embedding(max_features, 128),
        keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    return model

sentiment_model = build_sentiment_model(max_features, maxlen)
sentiment_model.summary()

# Train model
print('Training...')
history = sentiment_model.fit(x_train, y_train,
                             batch_size=32,
                             epochs=5,
                             validation_data=(x_test, y_test),
                             verbose=1)

# Evaluate
score, acc = sentiment_model.evaluate(x_test, y_test, batch_size=32, verbose=0)
print(f'Test accuracy: {acc:.4f}')

##3. Encoder-Decoder untuk Neural Machine Translation

###Teori Neural Machine Translation (NMT)
Neural Machine Translation adalah pendekatan end-to-end untuk machine translation menggunakan neural networks. Berbeda dengan statistical machine translation yang menggunakan multiple components terpisah, NMT menggunakan single neural network yang dilatih secara end-to-end.
<br><br>

###Evolusi Machine Translation:

- Rule-based MT: Menggunakan linguistic rules dan dictionaries
- Statistical MT: Menggunakan statistical models dari parallel corpora
- Neural MT: Menggunakan neural networks, khususnya sequence-to-sequence models
<br><br>

###Keuntungan NMT:

- End-to-end training: Tidak memerlukan alignment atau phrase extraction
- Better fluency: Menghasilkan terjemahan yang lebih natural
- Contextual understanding: Lebih baik dalam menangkap konteks
- Rare word handling: Lebih baik dalam menangani kata-kata jarang

Arsitektur Sequence-to-Sequence:
Seq2Seq model terdiri dari dua komponen utama:
<br><br>

###Encoder:

- Membaca input sequence dan mengkodekannya menjadi fixed-size context vector
- Biasanya menggunakan LSTM/GRU bidirectional
- Context vector adalah hidden state terakhir dari encoder
- Bertugas memampatkan seluruh informasi input ke dalam single vector


###Decoder:

- Menghasilkan output sequence dari context vector
- Menggunakan context vector sebagai initial hidden state
- Bekerja secara autoregressive (prediksi token berikutnya berdasarkan token sebelumnya)
- Training menggunakan teacher forcing untuk mempercepat konvergensi
<br><br>

###Masalah Information Bottleneck:
Arsitektur encoder-decoder basic memiliki masalah fundamental yaitu information bottleneck. Seluruh informasi dari input sequence harus dikompres ke dalam single context vector dengan ukuran tetap. Ini menyebabkan:

- Loss of information untuk sequence yang panjang
- Degradasi performance pada kalimat yang panjang
- Kesulitan dalam menangkap detail-detail penting
<br><br>

###Teacher Forcing:
Teknik training di mana decoder menggunakan ground truth dari timestep sebelumnya sebagai input, bukan prediksi model sendiri. Ini mempercepat training tetapi dapat menyebabkan exposure bias saat inference.

###Sequence-to-Sequence Architecture

In [ ]:
# Contoh implementasi Encoder-Decoder untuk terjemahan
def build_encoder_decoder_model(input_vocab_size, target_vocab_size,
                               embedding_dim=256, latent_dim=512):

    # Encoder
    encoder_inputs = keras.Input(shape=(None,))
    encoder_embedding = keras.layers.Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = keras.Input(shape=(None,))
    decoder_embedding = keras.layers.Embedding(target_vocab_size, embedding_dim)
    decoder_embedding_output = decoder_embedding(decoder_inputs)
    decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding_output, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

# Contoh penggunaan (perlu data preprocessing yang sesuai)
# model = build_encoder_decoder_model(input_vocab_size=10000, target_vocab_size=8000)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

##4. Attention Mechanism
###Teori Attention Mechanism
Attention mechanism adalah teknik yang memungkinkan model untuk "memperhatikan" bagian-bagian berbeda dari input sequence ketika menghasilkan setiap token output. Ini mengatasi masalah information bottleneck pada arsitektur encoder-decoder tradisional.
<br><br>

###Motivasi Attention:
Dalam terjemahan manusia, kita tidak mengingat seluruh kalimat input lalu menerjemahkannya. Sebaliknya, kita fokus pada bagian-bagian tertentu dari input ketika menerjemahkan setiap kata. Attention mechanism meniru proses ini.
<br><br>

###Prinsip Kerja Attention:

- Alignment: Menentukan bagian mana dari input yang relevan untuk output saat ini
- Weighting: Memberikan bobot pada setiap bagian input berdasarkan relevansinya
- Context: Membuat context vector berdasarkan weighted sum dari encoder outputs
<br><br>

###Jenis-jenis Attention:

###a. Bahdanau Attention (Additive Attention):

- Menggunakan feedforward network untuk menghitung alignment scores
- Score(h_i, s_j) = v^T tanh(W_h h_i + W_s s_j)
- Lebih fleksibel tetapi lebih lambat
<br><br>

###b. Luong Attention (Multiplicative Attention):

- Menggunakan dot product untuk menghitung alignment scores
- Score(h_i, s_j) = h_i^T W s_j (general) atau h_i^T s_j (dot)
- Lebih efisien tetapi kurang fleksibel
<br><br>

###c. Self-Attention:

- Attention dalam satu sequence (input dengan dirinya sendiri)
- Memungkinkan model memahami relationship antar posisi dalam sequence
- Basis dari Transformer architecture
<br><br>

###Keuntungan Attention:

- Mengatasi bottleneck: Tidak perlu kompres semua informasi ke single vector
- Better long sequences: Performa tidak menurun drastis pada sequence panjang
- Interpretability: Attention weights memberikan insight tentang alignment
- Selective focus: Model dapat fokus pada informasi yang relevan
<br><br>

###Attention Score Computation:

- Compute alignment scores: e_ij = align(s_{i-1}, h_j)
- Normalize scores: α_ij = softmax(e_ij)
- Compute context: c_i = Σ α_ij * h_j
<br><br>

###Multi-Head Attention:
Ekstensi dari attention mechanism yang memungkinkan model untuk memperhatikan informasi dari berbagai representasi subspaces secara bersamaan. Setiap "head" mempelajari jenis relationship yang berbeda.

###Bahdanau Attention

In [ ]:
class BahdanauAttention(keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)

    def call(self, query, values):
        # query shape: (batch_size, hidden_size)
        # values shape: (batch_size, max_len, hidden_size)

        # Expand query to (batch_size, 1, hidden_size)
        query_with_time_axis = tf.expand_dims(query, 1)

        # Calculate attention weights
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # Attention weights shape: (batch_size, max_len, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # Context vector shape: (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Encoder dengan Attention
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

# Decoder dengan Attention
class Decoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
        self.fc = keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))

        x = self.fc(output)

        return x, state, attention_weights

###Training Loop dengan Attention

In [ ]:
def train_step(inp, targ, enc_hidden, encoder, decoder, optimizer,
               loss_object, batch_size, targ_lang_tokenizer):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']] * batch_size, 1)

        # Teacher forcing
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_object(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)  # Teacher forcing

    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

# Fungsi untuk training
def train_model(dataset, encoder, decoder, epochs, steps_per_epoch):
    optimizer = tf.keras.optimizers.Adam()
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

    for epoch in range(epochs):
        enc_hidden = encoder.initialize_hidden_state()
        total_loss = 0

        for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
            batch_loss = train_step(inp, targ, enc_hidden, encoder, decoder,
                                  optimizer, loss_object, BATCH_SIZE, target_tokenizer)
            total_loss += batch_loss

            if batch % 100 == 0:
                print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')

        print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')

##5. Transformer Architecture (Preview)
###Teori Transformer
Transformer adalah arsitektur yang sepenuhnya bergantung pada attention mechanism tanpa menggunakan RNN atau CNN. Diperkenalkan dalam paper "Attention Is All You Need" (Vaswani et al., 2017), Transformer menjadi foundation untuk model-model modern seperti BERT, GPT, dan T5.
<br><br>

###Motivasi Transformer:

- Parallelization: RNN harus diproses secara sequential, sedangkan Transformer dapat diparalelkan
- Long-range dependencies: Self-attention dapat menangkap dependensi jangka panjang lebih baik
- Computational efficiency: Lebih efisien untuk sequence yang panjang
- Training speed: Dapat dilatih lebih cepat karena paralelisasi
<br><br>

###Komponen Utama Transformer:

###a. Self-Attention Mechanism:

- Query (Q), Key (K), Value (V) matrices
- Attention(Q,K,V) = softmax(QK^T/√d_k)V
- Memungkinkan setiap posisi memperhatikan semua posisi lain dalam sequence
<br><br>

###b. Multi-Head Attention:

- Multiple attention heads yang belajar representation yang berbeda
- Hasil dari semua heads digabungkan dan diproyeksikan
- Memungkinkan model fokus pada berbagai aspek relationship
<br><br>

###c. Position Encoding:

- Karena tidak ada urutan inherent dalam attention, perlu positional information
- Menggunakan sinusoidal encoding atau learned positional embeddings
- PE(pos, 2i) = sin(pos/10000^(2i/d_model))
- PE(pos, 2i+1) = cos(pos/10000^(2i/d_model))
<br><br>

###d. Feed-Forward Networks:

- Two-layer neural network dengan ReLU activation
- FFN(x) = max(0, xW1 + b1)W2 + b2
- Applied pada setiap posisi secara terpisah
<br><br>

###e. Residual Connections dan Layer Normalization:

- Residual connection: output = LayerNorm(x + Sublayer(x))
- Membantu training pada network yang dalam
- Layer normalization lebih stabil untuk sequence modeling
<br><br>

###Encoder-Decoder Structure:

- Encoder: Stack of identical layers, each with multi-head self-attention dan FFN
- Decoder: Stack dengan masked multi-head attention, encoder-decoder attention, dan FFN
- Masking: Mencegah decoder melihat future tokens selama training
<br><br>

###Keuntungan Transformer:

- Parallelizable: Semua posisi dapat diproses bersamaan
- Long-range dependencies: Self-attention menghubungkan posisi yang jauh
- Interpretable: Attention weights memberikan insight yang jelas
- Transfer learning: Pre-trained model dapat di-fine-tune untuk berbagai tugas
<br><br>

###Computational Complexity:

- Self-attention: O(n²d) dimana n adalah sequence length
- RNN: O(nd²) tetapi sequential
- Untuk sequence pendek dengan dimensi besar, RNN lebih efisien
- Untuk sequence panjang, Transformer lebih efisien dengan paralelisasi

###Self-Attention Mechanism

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
    """Calculate the attention weights."""
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    # Scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # Add mask to the scaled tensor
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # Softmax
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)

    return output, attention_weights

class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = keras.layers.Dense(d_model)
        self.wk = keras.layers.Dense(d_model)
        self.wv = keras.layers.Dense(d_model)

        self.dense = keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask=None):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)

        return output, attention_weights

##6. Text Preprocessing dan Tokenization
###Teori Text Preprocessing
Text preprocessing adalah langkah kritis dalam NLP yang menentukan kualitas input untuk model. Preprocessing yang baik dapat meningkatkan performa model secara signifikan, sementara preprocessing yang buruk dapat merusak informasi penting.
<br><br>

###Prinsip Dasar Preprocessing:

- Normalization: Menstandarkan format teks
- Tokenization: Memecah teks menjadi unit-unit yang dapat diproses
- Encoding: Mengkonversi token menjadi representasi numerik
- Sequence preparation: Menyiapkan sequence untuk model
<br><br>

###Tahapan Preprocessing:

###a. Text Cleaning:

- Case normalization: Lowercase/uppercase standardization
- Punctuation handling: Removal atau normalization
- Number handling: Replacement dengan placeholders
- Special characters: Removal atau normalization
- Unicode normalization: Menangani berbagai encoding
<br><br>

###b. Tokenization Strategies:

- Word-level: Memecah berdasarkan whitespace dan punctuation
- Subword-level: BPE, WordPiece, SentencePiece
- Character-level: Setiap karakter adalah token
- Hybrid approaches: Kombinasi berbagai strategi
<br><br>

###c. Vocabulary Management:

- Vocabulary size: Trade-off antara coverage dan efficiency
- OOV (Out-of-Vocabulary) handling: Unknown tokens, subword fallback
- Frequency thresholding: Menghilangkan token yang jarang
- Special tokens: <START>, <END>, <UNK>, <PAD>
<br><br>

###Tokenization Challenges:

- Ambiguous boundaries: Contractions, hyphenated words
- Multilingual: Berbagai script dan writing systems
- Domain-specific: Medical, legal, technical terminology
- Social media: Hashtags, mentions, emojis
- Morphologically rich languages: Agglutinative languages
<br><br>

###Subword Tokenization:
Mengatasi masalah OOV dan vocabulary size dengan memecah kata menjadi subword units:
<br><br>

###a. Byte Pair Encoding (BPE):

- Iteratively merge most frequent character pairs
- Balances between character-level dan word-level
- Dapat menangani morfologi dan OOV words
<br><br>

###b. WordPiece:

- Similar dengan BPE tetapi menggunakan likelihood-based merging
- Digunakan dalam BERT dan model Google lainnya
- Optimizes for language model perplexity
<br><br>

###c. SentencePiece:

- Treats text as sequence of Unicode characters
- Tidak memerlukan pre-tokenization
- Language-agnostic approach
<br><br>

###Sequence Preparation:

- Padding: Menyamakan panjang sequence dalam batch
- Truncation: Memotong sequence yang terlalu panjang
- Attention masks: Menandai posisi yang valid vs padding
- Special token insertion: Menambahkan token khusus untuk tugas tertentu

###Advanced Text Preprocessing

In [ ]:
import re
import unicodedata

def preprocess_sentence(w):
    # Convert to lowercase
    w = w.lower().strip()

    # Creating a space between a word and the punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # Replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.strip()

    # Adding start and end tokens
    w = '<start> ' + w + ' <end>'
    return w

def create_dataset(path, num_examples=None):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]
                  for l in lines[:num_examples]]

    return zip(*word_pairs)

# Tokenization
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

def load_dataset(path, num_examples=None):
    # Create dataset
    target_lang, input_lang = create_dataset(path, num_examples)

    input_tensor, input_lang_tokenizer = tokenize(input_lang)
    target_tensor, target_lang_tokenizer = tokenize(target_lang)

    return input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer

##7. Evaluasi Model NLP
###Teori Evaluasi NLP
Evaluasi model NLP berbeda dengan evaluasi model machine learning lainnya karena output berupa text yang memiliki karakteristik khusus. Evaluasi yang baik harus mempertimbangkan berbagai aspek seperti fluency, adequacy, dan semantic similarity.
<br><br>

###Tantangan dalam Evaluasi NLP:

- Subjektivity: Kualitas text sering subjektif
- Multiple correct answers: Satu input dapat memiliki berbagai output yang benar
- Context dependency: Kualitas bergantung pada konteks penggunaan
- Semantic vs syntactic: Evaluasi makna vs struktur
- Human evaluation cost: Evaluasi manusia mahal dan tidak scalable
<br><br>

###Jenis-jenis Metrik Evaluasi:

###a. Reference-based Metrics:
Memerlukan ground truth reference untuk comparison
###BLEU (Bilingual Evaluation Understudy):

- Mengukur n-gram precision antara candidate dan reference
- BLEU = BP × exp(Σ w_n log p_n)
- BP (Brevity Penalty) menghukum output yang terlalu pendek
- Baik untuk machine translation, kurang untuk generation tasks
- Kelemahan: tidak mempertimbangkan semantic similarity
<br><br>

###ROUGE (Recall-Oriented Understudy for Gisting Evaluation):

- Focus pada recall rather than precision
- ROUGE-N: n-gram recall
- ROUGE-L: Longest Common Subsequence
- ROUGE-W: Weighted LCS
- Cocok untuk summarization tasks
<br><br>

###METEOR (Metric for Evaluation of Translation with Explicit ORdering):

- Combines precision, recall, dan alignment
- Mempertimbangkan synonyms dan stemming
- Lebih korelasi dengan human judgment dibanding BLEU
<br><br>

###b. Embedding-based Metrics:
Menggunakan word/sentence embeddings untuk semantic comparison
###BERTScore:

- Menggunakan BERT embeddings untuk token-level similarity
- Menghitung precision, recall, dan F1 pada embedding space
- Lebih sensitive terhadap semantic meaning
<br><br>

###Sentence-BERT:

- Menggunakan sentence-level embeddings
- Cosine similarity antara candidate dan reference embeddings
<br><br>

###c. Task-specific Metrics:

- Sentiment Analysis: Accuracy, Precision, Recall, F1
- Named Entity Recognition: Entity-level F1, exact match
- Question Answering: Exact Match, F1 on tokens
- Text Classification: Standard classification metrics
<br><br>


###Human Evaluation:

- Fluency: Seberapa natural dan grammatical text yang dihasilkan
- Adequacy: Seberapa baik meaning dari source dipertahankan
- Relevance: Seberapa relevan output dengan input/context
- Coherence: Konsistensi internal dalam text
- Inter-annotator agreement: Konsistensi antar evaluator
<br><br>

###Evaluation Best Practices:

- Multiple metrics: Gunakan berbagai metrik untuk comprehensive evaluation
- Statistical significance: Test significance dari improvement
- Error analysis: Analisis kualitatif dari errors
- Domain evaluation: Test pada berbagai domain
- Robustness testing: Evaluation pada adversarial examples
<br><br>

###Challenges dengan Reference-based Metrics:

- Single reference limitation: Real-world tasks sering memiliki multiple valid outputs
- Surface-level comparison: Metrics seperti BLEU hanya compare surface forms
- Gaming metrics: Model dapat dioptimasi untuk metric tanpa improving actual quality
- Context ignorance: Tidak mempertimbangkan context yang lebih luas

###BLEU Score untuk Machine Translation

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

def evaluate_translation(encoder, decoder, sentence, input_lang_tokenizer, target_lang_tokenizer, max_length):
    attention_plot = np.zeros((max_length, max_length))

    sentence = preprocess_sentence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)

    for t in range(max_length):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += target_lang_tokenizer.index_word[predicted_id] + ' '

        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# Calculate BLEU score
def calculate_bleu_score(references, candidates):
    """
    Calculate BLEU score for a set of reference and candidate sentences
    """
    bleu_score = corpus_bleu(references, candidates)
    return bleu_score

##Kesimpulan
Chapter 16 membahas aplikasi RNN dan Attention untuk NLP:

- Character-Level Language Models: Untuk text generation sederhana
- Sentiment Analysis: Menggunakan LSTM untuk klasifikasi teks
- Neural Machine Translation: Encoder-decoder dengan attention
- Attention Mechanism: Meningkatkan performa pada sekuens panjang
- Text Preprocessing: Teknik preprocessing yang robust
- Evaluation Metrics: BLEU score dan metrik evaluasi NLP lainnya
<br><br>

###Key Takeaways:

- RNN cocok untuk tugas NLP yang membutuhkan pemahaman konteks
- Attention mechanism sangat penting untuk tugas seq2seq
- Preprocessing yang baik sangat kritikal untuk performa model
- Evaluasi model NLP membutuhkan metrik khusus seperti BLEU
- Transformer (preview) menunjukkan arah masa depan NLP
<Br><br>

###Tips Praktis:

- Gunakan teacher forcing untuk training yang lebih stabil
- Implementasikan attention untuk tugas translation
- Lakukan preprocessing yang konsisten antara training dan inference
- Monitor attention weights untuk interpretability
- Pertimbangkan model pre-trained untuk tugas praktis